In [1]:
from __future__ import division
import shutil as sh
from os.path import join 
from glob import glob
import itertools
from distutils.dir_util import copy_tree
import shutil as shutil
from glob import glob
import pandas as pd

import numpy as np
from subprocess import check_call
import matplotlib.pyplot as plt
#from utilities import ensure_dir, get_n_vols
from pylab import plot, ylim, xlim, show, xlabel, ylabel, grid
from numpy import linspace, loadtxt, ones, convolve
from nipype.interfaces.dcm2nii import Dcm2niix, Dcm2nii
#import nipype_init_new
#nipype_init_new.init()
from nilearn.connectome import sym_matrix_to_vec, vec_to_sym_matrix,ConnectivityMeasure
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img, plot_matrix, plot_glass_brain
from nilearn.image import concat_imgs, mean_img, load_img, index_img, math_img, resample_img
from nilearn.masking import apply_mask
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img
from nilearn import plotting
from statsmodels.sandbox.stats.multicomp import multipletests as multi
from scipy.stats import norm
import statsmodels.stats.multitest as ssm
import statsmodels
import seaborn as sns
from scipy.stats import pearsonr,ttest_ind
from sklearn.model_selection import StratifiedKFold,permutation_test_score
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
import time

/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# migrate functions in nistats to nilearn
# https://nilearn.github.io/nistats_migration.html#design-matrix
from nilearn.glm.first_level import FirstLevelModel
#from nistats.thresholding import map_threshold # can not be migrate directly
from nilearn.glm import threshold_stats_img
from nilearn.reporting import get_clusters_table
from nilearn.plotting import plot_contrast_matrix, plot_design_matrix
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.glm.second_level import SecondLevelModel
from nltools.data import Brain_Data

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def ensure_dir(path):
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)


In [5]:
topdir="/media/haiyanwu/HDD2/fmri2019/"

roi_dir=topdir+'3.ROIextraction/'
mask_dir=roi_dir+'masks/'



## ROI extraction: using neurosynth masks

In [6]:
ROIs=['ANG','BA6-SMA','BA8-preSMA','BA9-dlPFC','BA13-insula','BA24-dACC','BA28-hip1','BA34-hip','BA35-hip2','cc_z5','cc_z10',
      'reward_z10','reward_z5','rewant_z10','rewant_z5','moral_z10','moral_z5','monrew_z10','monrew_z5','memory_z10','memory_z5',
     'maintenance_z10','maintenance_z5','lDLPFC','rDLPFC','ventral_striatum','putamen']
len(ROIs)


27

In [7]:
subList=[102,103,105,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,127,128,129,130,131,132]
len(subList)

26

In [12]:
# extract from GLM1 : contrast1-9
glm_dir=topdir+'2.GLM/3.GLM_SPM/results/GLM1/'
writer = pd.ExcelWriter('ROIbeta_GLM1.xlsx', engine='xlsxwriter')
for indroi, roi in enumerate(ROIs):
    mask=Brain_Data(mask_dir+'Mask_%s.nii'%(roi))
    betas=[]
    for indsub, subno in enumerate(subList):
 
        subbeta=[]
        for runind,runno in enumerate([1,2,3,4,5,6,7,8,9,10,11,12]):
            
            temp=Brain_Data(glm_dir+'sub-%03d/con_%04d.nii'%(subno,runno))
            masked=temp.apply_mask(mask)
            subbeta.append(masked.mean())
        betas.append(subbeta)
    betas=pd.DataFrame(betas,index=subList)
    betas.to_excel(writer, index=subList,header=[1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12], sheet_name='%s' % roi, startcol=1)
           
writer.save()      

In [8]:
# extract from GLM2 : contrast1-9
glm_dir=topdir+'2.GLM/3.GLM_SPM/results/GLM2/'
writer = pd.ExcelWriter('ROIbeta_GLM2.xlsx', engine='xlsxwriter')
for indroi, roi in enumerate(ROIs):
    mask=Brain_Data(mask_dir+'Mask_%s.nii'%(roi))
    betas=[]
    for indsub, subno in enumerate(subList):
 
        subbeta=[]
        for runind,runno in enumerate([1,2,3,4,5,6,7,8,9,10,11,12]):
            
            temp=Brain_Data(glm_dir+'sub-%03d/con_%04d.nii'%(subno,runno))
            masked=temp.apply_mask(mask)
            subbeta.append(masked.mean())
        betas.append(subbeta)
    betas=pd.DataFrame(betas,index=subList)
    betas.to_excel(writer, index=subList,header=[1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12], sheet_name='%s' % roi, startcol=1)
           
writer.save()      